In [1]:
# coding: utf-8
"""
gensimを用いたLDA
purityの計算
"""

from gensim import corpora, models, similarities
import csv
import MeCab
import pickle
import collections
import numpy as np

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)

### コーパスの読み込み
* 1行ずつ形態素解析して、ラベルが付いたものと、口コミごとに形態素解析しているのものの2種類を読み込む

In [2]:
list_sep_words = readcsv("./files/rakuten_corpus/rakuten_corpus_sep.csv")

### LDAにより、トピックの抽出
1. 辞書の作成
2. 辞書を用いてコーパスのマッピングをする
3. マッピングしたコーパスを元にLDA
4. LDAの結果を表示

In [3]:
# 辞書の作成
dictionary = corpora.Dictionary(list_sep_words)
# コーパスをマッピングし、LDA
corpus = [dictionary.doc2bow(text) for text in list_sep_words]
lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=7)
# p(word|topic)を表示
for i, row in enumerate(lda.show_topics(0)):
    print row

0.028*大変 + 0.019*駅 + 0.016*朝食 + 0.015*よい + 0.014*近い + 0.013*便利 + 0.013*プラン + 0.011*食事 + 0.011*対応 + 0.011*綺麗
0.026*朝食 + 0.020*安い + 0.014*フロント + 0.012*よい + 0.012*大変 + 0.012*近い + 0.012*対応 + 0.010*温泉 + 0.010*感じ + 0.010*ない
0.019*食事 + 0.018*満足 + 0.017*風呂 + 0.017*サービス + 0.017*対応 + 0.014*立地 + 0.013*広い + 0.011*感じ + 0.011*よい + 0.010*予約
0.024*対応 + 0.022*フロント + 0.021*朝食 + 0.017*よい + 0.013*広い + 0.011*満足 + 0.011*食事 + 0.011*残念 + 0.010*美味しい + 0.009*料金
0.018*朝食 + 0.017*ない + 0.016*満足 + 0.014*残念 + 0.014*広い + 0.012*風呂 + 0.011*対応 + 0.011*大変 + 0.011*快適 + 0.011*食事
0.036*朝食 + 0.018*満足 + 0.015*駅 + 0.014*近い + 0.013*よい + 0.013*立地 + 0.012*風呂 + 0.012*対応 + 0.011*便利 + 0.011*いい
0.021*満足 + 0.019*よい + 0.017*サービス + 0.016*立地 + 0.015*朝食 + 0.014*ない + 0.014*フロント + 0.012*対応 + 0.010*スタッフ + 0.010*出張


### 文の予測
1. LDAの結果から各文のクラスを予測
2. 予測ラベルと結果ラベルのリストを作成

In [16]:
list_predict_measure = []
error_count = 0
for i in range(len(corpus_1)):
    predict_class = sorted(lda[corpus_1[i]], key=lambda x:x[1], reverse=True)[0][0]
    measure_class = list_class[i]
    try:
        list_predict_measure.append([predict_class, int(measure_class)])
    except:
        error_count += 1
print "エラーしたコーパス", error_count

エラーしたコーパス 7


In [17]:
list_predict, list_class = zip(*list_predict_measure)
collections.Counter(list(list_predict))

Counter({1: 963, 6: 596, 2: 457, 4: 407, 0: 355, 3: 311, 5: 265})

### 定量評価 (purity)

In [18]:
cal_f_measure(list_predict_measure)

Purity:  0.243291592129
Inverse Purity:  0.287119856887
F-value:  0.263394944598
